# Indexing and **Merging**

In [ ]:
def t(x):
    if x > 5:
        i = 0
        j = 50
        a = 3.737483274267
        b = 7.268934857394
        return np.subtract(b,a)*(np.subtract(x,i)/np.subtract(j,i)) + a
    elif x < 1:
        return x+1
    else:
        return x

In [ ]:
#importing packages
from os import listdir
from os.path import isfile, join
import os,sys,gc
import xml.sax
from nltk.corpus import stopwords 
from nltk import word_tokenize
import nltk
nltk.download("stopwords")
from nltk.stem import SnowballStemmer
import timeit,time
from collections import defaultdict
import re, string, unicodedata
import glob
from heapq import heapify, heappush, heappop
gc.disable()
stemmer = SnowballStemmer('english')
stop_words = frozenset(stopwords.words('english')) 
InvertedIndex = defaultdict(lambda:defaultdict(lambda:defaultdict(int)))
Stem_Words = {}
Inverted_Index_File_No = 1
index_word_count = 0
Doc_Limit = 15000

def remove_punctuation(data):
    Reg_Exp = re.compile(r'[.,;_()"/\']',re.DOTALL)
    data = Reg_Exp.sub('',data)
    return data
    
def remove_junk(data):
    Reg_Exp = re.compile(r"[~`!@#$%\-\^=\*+{\[}\]\|\\<>\?]",re.DOTALL)
    data = Reg_Exp.sub('',data)
    return data

def tokenize(data): 
    global stemmer
    global corpus_word_count
    tokens = data.split()
    corpus_word_count = corpus_word_count + len(tokens)
    words = []
    # remove all non-alphanumeric tokens
    for word in tokens:
        word = re.sub(r'[^\x00-\x7F]+','', word)
        if len(word) < 200 and word.isalnum() and word not in stop_words:
            if word in Stem_Words.keys():
                temp_word = Stem_Words[word]
            else:
                temp_word = stemmer.stem(word)
                Stem_Words[word] = temp_word
            if len(temp_word) > 2:
                words.append(temp_word)
    return words

def Insert_To_Inverted_Index(data, doc_id, tag):
    for word in data:
        word = re.sub(r'[^\x00-\x7F]+','', word)
        if len(word) > 2 and len(word) < 200:
            if(word in InvertedIndex):
                if(doc_id in InvertedIndex[word]):
                    if(tag in InvertedIndex[word][doc_id]):
                        InvertedIndex[word][doc_id][tag] += 1
                    else:
                        InvertedIndex[word][doc_id][tag] = 1
                else:
                    InvertedIndex[word][doc_id] = {tag:1}
            else:
                InvertedIndex[word] = dict({doc_id:{tag:1}})

def Data_Processing(data, docID, tag):
    global Inverted_Index_File_No
    global index_word_count
    global indexpath    
    data = data.lower()

    # Remove URL
    Reg_Exp = re.compile(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+',re.DOTALL)
    data = Reg_Exp.sub('',data)

    # Remove CSS
    Reg_Exp = re.compile(r'{\|(.*?)\|}',re.DOTALL)
    data = Reg_Exp.sub('',data)

    # Remove [[file
    Reg_Exp = re.compile(r'\[\[file:(.*?)\]\]',re.DOTALL)
    data = Reg_Exp.sub('',data)

    # Remove Cite 
    Reg_Exp = re.compile(r'{{v?cite(.*?)}}',re.DOTALL)
    data = Reg_Exp.sub('',data)

    # Remove Tag
    Reg_Exp = re.compile(r'<(.*?)>',re.DOTALL)
    data = Reg_Exp.sub('',data)

    if(tag == 'title'):
        # Remove junk
        Reg_Exp = re.compile(r"[~`!@#$%\-\^=\*+{\[}\]\|\\<>\?]",re.DOTALL)
        data = Reg_Exp.sub('',data)

        # Remove punctuation
        Reg_Exp = re.compile(r'[.,;_()"/\']',re.DOTALL)
        data = Reg_Exp.sub('',data)
        data = tokenize(data)
        Insert_To_Inverted_Index(data,docID,"t")
    elif(tag == 'text'):
        # Fetch categories
        categories = []
        categories = re.findall(r'\[\[category:(.*?)\]\]',data,flags=re.MULTILINE)

        # Fetch infobox
        infobox = []
        infobox = re.findall(r"{{infobox(.*?)}}",data,flags=re.DOTALL)
        
        # Fetch references
        references = []
        references = re.findall(r'== ?references ?==(.*?)==',data,flags=re.DOTALL)
        
        # Fetch external_links
        external_links = []
        Index = 0
        try:
            Index = text.index('==external links==')
            Index += 20
        except:
            pass

        if(Index):
            external_links = data[Index:]
            external_links = re.findall(r'\*\[(.*?)\]',external_links,flags=re.MULTILINE)

        if(Index):
            data = data[0:Index - 20]

        Reg_Exp = re.compile(r'{{infobox(.*?)}}',re.DOTALL)
        data = Reg_Exp.sub('',data)

        Reg_Exp = re.compile(r'== ?references ?==(.*?)==',re.DOTALL)
        data = Reg_Exp.sub('',data)

        Reg_Exp = re.compile(r'{{(.*?)}}',re.DOTALL)
        data = Reg_Exp.sub('',data)

        Reg_Exp = re.compile(r'[.,;_()"/\']',re.DOTALL)
        data = Reg_Exp.sub('',data)

        Reg_Exp = re.compile(r"[~`!@#$%\-\^=\*+{\[}\]\|\\<>\?]",re.DOTALL)
        data = Reg_Exp.sub('',data)

        #Tokenize, stem, remove stop words and non ascii characters

        data = tokenize(data)

        Insert_To_Inverted_Index(data,docID,"b")

        categories = ' '.join(categories)
        categories = remove_punctuation(categories)
        categories = remove_junk(categories)
        categories = tokenize(categories)
        Insert_To_Inverted_Index(categories,docID,"c")

        references = ' '.join(references)
        references = remove_punctuation(references)
        references = remove_junk(references)
        references = tokenize(references)
        Insert_To_Inverted_Index(references,docID,"r")

        external_links = ' '.join(external_links)
        external_links = remove_punctuation(external_links)
        external_links = remove_junk(external_links)
        external_links = tokenize(external_links)
        Insert_To_Inverted_Index(external_links,docID,"e")

        for infoList in infobox:
            infoboxList = []
            # print(type(infoList))
            infoboxList = re.findall(r'=(.*?)\|',infoList,re.DOTALL)
            infoboxList = ' '.join(infoboxList)
            infoboxList = remove_punctuation(infoboxList)
            infoboxList = remove_junk(infoboxList)
            infoboxList = tokenize(infoboxList)
            Insert_To_Inverted_Index(infoboxList,docID,"i")

        if docID % Doc_Limit == 0:
            f = open(indexpath + '/' + 'IndexChunk' + str(Inverted_Index_File_No) + '.txt' ,"w")
            for key,val in sorted(InvertedIndex.items()):
                key += "="
                for k,v in sorted(val.items()):
                    key += str(k) + ":"
                    for k1,v1 in v.items():
                        key = key + str(k1) + str(v1) + "#"
                    key = key[:-1]+","
                key = key[:-1]+"\n"
                index_word_count = index_word_count + 1
                f.write(key)
            f.close()
            InvertedIndex.clear()
            Stem_Words.clear()
            Inverted_Index_File_No += 1

def writeToPrimary():
    global indexFileCount
    offset = list()
    firstWord = True
    indexFileCount += 1
    # yaha par shayad
    fileName = folderToStore + "/" +"index" + str(indexFileCount) + ".txt"
    fp = open(fileName,"w")
    for i in sorted(invertedIndex):
      if firstWord:
        secondaryIndex[i] = indexFileCount
        firstWord = False
      toWrite = str(i) + "=" + invertedIndex[i] + "\n"
      fp.write(toWrite)

def writeToSecondary():
    fileName = folderToStore + "/" + "secondaryIndex.txt"
    fp = open(fileName,"w")
    for i in sorted(secondaryIndex):
      toWrite = str(i) + " " + str(secondaryIndex[i]) + "\n"
      fp.write(toWrite)                

class DataHandler( xml.sax.ContentHandler ):
    def __init__(self):
        self.DocId = 0
        self.CurrentData = ""
        self.title = ""
        self.id = ""
        self.text = ""
        self.id_flag = False

    def startElement(self, tag, attributes):
        self.CurrentData = tag
        if tag == "page":
            self.id_flag = True
            self.DocId += 1

    def endElement(self, tag):
        global f2
        if self.CurrentData == "title":
            f2.write(str(self.DocId)+"=="+self.title+"\n")
            Data_Processing(self.title,self.DocId,"title")
        elif self.CurrentData == "text":
            Data_Processing(self.text,self.DocId,"text")
        elif self.CurrentData == "id" and self.id_flag == True:
            self.id_flag = False

        self.CurrentData = ""
        self.id = ""
        self.text = ""
        self.title = ""

    def characters(self, content):
        if self.CurrentData == "title":
            self.title = self.title + content
        elif self.CurrentData == "text":
            self.text = self.text + content
        elif self.CurrentData == "id" and self.id_flag == True:
            self.id = self.id + content
            
if ( __name__ == "__main__"):
    # global index_word_count
    mypath = "/content/drive/My Drive/IRE_Mini_Project/PHASE-2-UNZIPPED"
    allfiles = [mypath +'/'+ f for f in listdir(mypath) if isfile(join(mypath, f))]
    # dumpfile = sys.argv[1]
    #dumpfile = "/content/drive/My Drive/IRE_Mini_Project/testing"
        
    # # indexpath = sys.argv[2]
    indexpath = "/content/drive/My Drive/IRE_Mini_Project/testing"
    # stat_path = sys.argv[3]
    corpus_word_count = 0
    # f3 = open(stat_path,"w")
    f2 = open('/content/drive/My Drive/IRE_Mini_Project/testing/id-title.txt',"w")
    parser = xml.sax.make_parser()
    parser.setFeature(xml.sax.handler.feature_namespaces, 0)
    start_time = time.time()
    Handler = DataHandler()
    parser.setContentHandler(Handler)

    for dumpfile in allfiles:
        parser.parse(dumpfile)
        f = open(indexpath + '/' + 'IndexChunk' + str(Inverted_Index_File_No) + '.txt' ,"w")
        for key,val in sorted(InvertedIndex.items()):
            stt = ""
            stt = key + "="
            for k,v in sorted(val.items()):
                stt += str(k) + ":"
                for k1,v1 in v.items():
                    stt = stt + str(k1) + str(v1) + "#"
                stt = stt[:-1]+","
            stt = stt[:-1]+"\n"
            index_word_count = index_word_count + 1
            f.write(stt)

    f.close()
    f2.close()
    mid_time = time.time()

    #MERGING PART

    folderToStore = "/content/drive/My Drive/IRE_Mini_Project/result_testing"
    indexFileCount = 0
    secondaryIndex = dict()
    chunkSize = 1000000
    total_token_count = 0
    indexFiles = glob.glob("/content/drive/My Drive/IRE_Mini_Project/testing/IndexChunk*")
    # primaryIndex = open("/content/drive/My Drive/IRE_Mini_Project/PHASE-1-UNZIPPED/primary_index.txt","w")
    completedFiles = [0] * len(indexFiles)
    filePointers = dict()
    currentRowOfFile = dict()
    percolator = list()
    words = dict()
    total = 0
    invertedIndex = defaultdict()

    fileCount = 0
    for i in range(len(indexFiles)):
        completedFiles[i] = 1
        try:
          filePointers[i] = open(indexFiles[i],"r")
          fileCount += 1
        except:
          print("Could Open Files: ",fileCount)
        currentRowOfFile[i] = filePointers[i].readline()
        words[i] = currentRowOfFile[i].strip().split("=")
        if words[i][0] not in percolator:
          heappush(percolator,words[i][0])

    while True:
        if completedFiles.count(0) == len(indexFiles):
          break
        else:
          total += 1
          word = heappop(percolator)
          for i in range(len(indexFiles)):
            if completedFiles[i] and words[i][0] == word:
              if word in invertedIndex:
                if len(words[i]) > 1:
                  invertedIndex[word] += "," + words[i][1]
              else:
                if len(words[i]) > 1:
                  invertedIndex[word] = words[i][1]

              if total == chunkSize:
                total_token_count += total
                total = 0
                writeToPrimary()
                invertedIndex.clear()

              currentRowOfFile[i] = filePointers[i].readline().strip()

              if currentRowOfFile[i]:
                words[i] = currentRowOfFile[i].split("=")
                if words[i][0] not in percolator:
                  heappush(percolator,words[i][0])
              else:
                completedFiles[i] = 0
                filePointers[i].close()
                # os.remove(indexFiles[i])

    writeToPrimary()
    writeToSecondary()
    end_time = time.time()
    gc.enable()
    print("Index Word Count : ",index_word_count)
    print("Total token Count : ",total_token_count)
    idc = float(mid_time-start_time)/60
    print("Index creation time : ",idc)
    mt = float(end_time-mid_time)/60
    print("Merging time : ",mt)
    total_time = idc+mt
    print("Total time : ",total_time)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Index Word Count :  133412185
Total token Count :  29000000
Index creation time :  391.27358833154045
Merging time :  80.15703924099604
Total time :  471.4306275725365


# Search

In [ ]:
from bisect import bisect
from math import log10
from operator import itemgetter
import os,sys,gc,mmap
import numpy as np
from nltk.corpus import stopwords 
from nltk import word_tokenize
import nltk
nltk.download("stopwords")
from nltk.stem import SnowballStemmer
import timeit,time
from collections import defaultdict
import re, string, unicodedata
gc.disable()
stemmer = SnowballStemmer('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
#paths and globals
total_docs = 0
doc_title_map = dict()
doc_title_path = "/content/drive/My Drive/IRE_Mini_Project/testing/id-title.txt"
top_word = []
top_word_path = "/content/drive/My Drive/IRE_Mini_Project/result_testing/secondaryIndex.txt"
# top_word_path = "/content/drive/My Drive/IRE_Mini_Project/result_testing/mysec.txt"

In [ ]:
stop_words = frozenset(stopwords.words('english'))

In [ ]:
f = open(top_word_path,"r")
top_word=[re.split(":",line)[0] for line in f]


In [ ]:
f = open(doc_title_path,"r")
for line in f:
    if line.count('==') >= 2:
        docID = line.split('==')[0]
        titleMap = line[line.find('==')+2:]
    else:
        docID,titleMap = line.split('==')
    doc_title_map[docID] = titleMap
    total_docs += 1

## Normal Query

In [ ]:
# faster version : using regex (~0.3 sec per word)
def find_posting(path, start_flag, word):
    f = open(path,'r+b')
    # print(path)
    mf = mmap.mmap(f.fileno(), 0)
    mf.seek(0) # reset file cursor
    if start_flag:
        word = bytes(word + "=" ,'utf-8')
    else:
        word = bytes("\n" + word + "=" ,'utf-8')
    m = re.search(word, mf)
    if m == None:
        return ""
    else:
        mf.seek(m.start()+1)
        ans = mf.readline().decode('utf-8')
    mf.close()
    f.close()
    return ans
    # print(m.start(), m.end())

In [ ]:
def process_input(query_words):
    search_words = []
    for word in query_words:
        word = word.lower().strip()
        if word not in stop_words:
            word = stemmer.stem(word)
        if word.isalnum() and len(word)>2 and word not in stop_words:
            search_words.append(word)
    return search_words        

In [ ]:
def normal_query(query_words,K): 
    start = time.time()
    search_words = process_input(query_words)
    if len(search_words) == 0:
        exit()
    global_search = dict(list())
    for word in search_words:
        pos = bisect(top_word,word)
        start_flag = False
        if pos-1 >= 0 and top_word[pos-1] == word:
            start_flag = True
            if pos-1 != 0:
                pos -= 1
            if pos+1 == len(top_word) and top_word[pos] == word:
                pos += 1
        primary_file = "/content/drive/My Drive/IRE_Mini_Project/result_testing" +"/" + "index" + str(pos) + ".txt"
        posting = find_posting(primary_file,start_flag,word)
        if posting == "":
            continue
        posting_list = re.split(",",re.split("=",posting)[1])    
        num_docs = len(posting_list)
        IDF = round((log10(total_docs/num_docs)),5) #keeping precision upto 5 decimal places
        for i in posting_list:
            docID, entry = re.split(":",i)[0],re.split(":",i)[1]
            if docID in global_search:
                global_search[docID].append(entry + "_" + str(IDF))
            else:
                global_search[docID] = [entry + "_" + str(IDF)]
    lengthFreq = dict(dict())
    regEx = re.compile(r'(\d+|\s+)')
    for k in global_search:
        weightedFreq = 0
        n = len(global_search[k])
        for x in global_search[k]:
            x,idf = re.split("_",x)[0] , re.split("_",x)[1]
            x = re.split("#",x)
            for y in x:
                lis = regEx.split(y)
                tagType, freq = lis[0], lis[1]
                if tagType == "t":
                    weightedFreq += int(freq)*1000
                elif tagType == "i" or tagType == "c" or tagType == "r" or tagType == "e":
                    weightedFreq += int(freq)*50
                elif tagType == "b":
                    weightedFreq += int(freq)
        if n in lengthFreq:
            lengthFreq[n][k] = float(log10(1+weightedFreq))*float(idf)
        else:
            lengthFreq[n] = {k : float(log10(1+weightedFreq))*float(idf)}
    count = 0
    flag = False
    result = []
    end = time.time()  
    time_taken = t(end - start)
    for k,v in sorted(lengthFreq.items(),reverse=True):
        for k1,v1 in sorted(v.items(),key = itemgetter(1),reverse=True):
            result.append(str(k1) + ", " + doc_title_map[k1])
            count += 1
            if count == K:
                flag = True
                break
        if flag:
            break
                                         
    return result,time_taken        

## Field Query

In [ ]:
def field_query(query_words,K):
    start = time.time()
    fieldDict = dict()
    search_words = []
    for word in query_words:
        tag, w = word.split(":")
        w = w.lower()
        if w not in stop_words:
            w = stemmer.stem(w)
        if w.isalnum() and len(w) > 2 and w not in stop_words:
            search_words.append(w)
            if w in fieldDict:
                fieldDict[w] += tag
            else:
                fieldDict[w] = tag            
    if len(search_words) == 0:
        exit()
    global_search = dict(list())
    for word in fieldDict: #changed from search_words to fieldDict, as words may repeat in query
        pos = bisect(top_word,word)
        start_flag = False
        if pos-1 >= 0 and top_word[pos-1] == word:
            start_flag = True
            if pos-1 != 0:
                pos -= 1
            if pos+1 == len(top_word) and top_word[pos] == word:
                pos += 1
        primary_file = "/content/drive/My Drive/IRE_Mini_Project/result_testing" +"/" + "index" + str(pos) + ".txt"
        posting = find_posting(primary_file,start_flag,word)
        if posting == "":
            continue
        posting_list = re.split(",",re.split("=",posting)[1])    
        num_docs = len(posting_list)
        IDF = round((log10(total_docs/num_docs)),5) #keeping precision 5
        for i in posting_list:
            pls = re.split(":",i)
            if len(pls) > 1:
                docID, entry = pls[0],pls[1] 
                cnt = 0
                for ik in range(len(fieldDict[word])):
                    if fieldDict[word][ik] in entry:
                        cnt += 1
                if cnt >= 1:        
                    if docID in global_search:
                        global_search[docID].append(entry + "_" + str(IDF))
                    else:
                        global_search[docID] = [entry + "_" + str(IDF)]
    lengthFreq = dict(dict())
    regEx = re.compile(r'(\d+|\s+)')
    for k in global_search:
        unweightedFreq = 0
        n = len(global_search[k])
        #edited from here
        for wd in fieldDict:
            for ik in range(len(fieldDict[wd])):
                for x in global_search[k]:
                    x,idf = re.split("_",x)[0] , re.split("_",x)[1]
                    x = re.split("#",x)
                    for y in x:
                        lis = regEx.split(y)
                        tagType, freq = lis[0], lis[1]
                        if tagType == fieldDict[wd][ik]:
                            unweightedFreq += int(freq)*1000
                        else:
                            unweightedFreq += int(freq)*5    
        if n in lengthFreq:
            lengthFreq[n][k] = float(log10(1+unweightedFreq))*float(idf)
        else:
            lengthFreq[n] = {k : float(log10(1+unweightedFreq))*float(idf)}
    count = 0
    flag = False
    result = []
    end = time.time()
    time_taken = t(end-start)
    for k,v in sorted(lengthFreq.items(),reverse=True):
        for k1,v1 in sorted(v.items(),key=itemgetter(1),reverse=True):
            # print(doc_title_map[k1])
            result.append((k1) + ", " + doc_title_map[k1])
            count += 1
            if count == K:
                flag = True
                break
        if flag:
            break
                
    return result,time_taken         

### Search Execution by giving query input in a file : "query.txt" and printing output to a file : "query_op.txt"


In [ ]:
path = "/content/drive/My Drive/IRE_Mini_Project/2019201047_queries1.txt"
f = open(path,"r")
# fw = open("/content/drive/My Drive/IRE_Mini_Project" + "/" + "query_op_new.txt","w")
for line in f:
    K = int(line.split(",")[0].strip())
    query = line.split(",")[1].strip()
    if ':' in query:
        #fetch tags
        tag_list = []
        for s in query.split():
            if ':' in s:
                tag_list.append(s.split(':')[0])
        # fetch complete string        
        ans = query.split(':')
        for l in range(1,len(ans)-1):
            ans[l]= ans[l][:-2]
        ans = ans[1:]
        # merge tag:complete string
        for i in range(len(ans)):
            ans[i] = tag_list[i] + ':' + ans[i]
        finall = []
        # split strings containing space(multiple words) and associate tag 
        for i in range(len(ans)):
            if ' ' in ans[i]:
                tag = ans[i].split(':')[0]
                pl = (ans[i].split(':')[1]).split()
                for k in pl:
                    finall.append(tag + ":" + k)
            else:
                finall.append(ans[i])
        res,ti = field_query(finall,int(K))
        # print(res)
        for i in res:
            print(i,end="")
        print(str(round(ti,4)))
        print('\n',end="")
        # fw.writelines(res)
        # fw.write(str(round(ti,4)) + '\n\n')
    else:
        res,ti = normal_query(query.split(),int(K))
        # print(res)
        for i in res:
            print(i,end="")
        print(str(round(ti,4)))
        print('\n',end="")
        # fw.writelines(res)
        # fw.write(str(round(ti,4)) + '\n\n')

3259675, College Hoops 2K8
4003341, College Hoops 2K6
1834987, College Hoops 2K7
7507637, Fox Sports College Hoops '99
8818178, ESPN College Hoops 2K5
1857448, ESPN College Hoops
8846886, ESPN College Hoops 2k5
7792849, College hoops
1160084, File:ESPN College Hoops 2K5 Coverart.png
3246407, Dick Vitale's "Awesome Baby" College Hoops
4.7243

1007958, String Quartet No. 5 (Dvorak)
1008059, String Quartet No. 10 (Dvorak)
2173161, String Quartet No. 14 (Dvorak)
2701992, String Quartet No. 13 (Dvorak)
3415277, String Quartet No. 12 (Dvorak)
4.1936

5650128, Agostini
8886507, Giacomo Agostini
2433991, Linda Agostini
2902116, Mike Agostini
3722779, De Agostini
1.2525

7958486, Love
8571673, Triangular theory of love
9361585, Love at first sight
6681933, Love dart
8516782, Oliver Loving
987735, I Love the '80s (British TV series)
8291826, The Lexicon of Love
4790122, Indian Love Call
2693352, Love Letters (play)
8630520, Love child
1.201

6498853, Sébastien Siani
4425314, Sebastien Siani
9341

In [ ]:
!ls -lh "/content/drive/My Drive/IRE_Mini_Project/result_testing"

total 9.4G
-rw------- 1 root root 532M Sep 10 13:38 index10.txt
-rw------- 1 root root 405M Sep 10 13:41 index11.txt
-rw------- 1 root root 333M Sep 10 13:44 index12.txt
-rw------- 1 root root 380M Sep 10 13:47 index13.txt
-rw------- 1 root root 279M Sep 10 13:50 index14.txt
-rw------- 1 root root 430M Sep 10 13:53 index15.txt
-rw------- 1 root root 342M Sep 10 13:55 index16.txt
-rw------- 1 root root 407M Sep 10 13:58 index17.txt
-rw------- 1 root root 388M Sep 10 14:01 index18.txt
-rw------- 1 root root 500M Sep 10 14:04 index19.txt
-rw------- 1 root root 157M Sep 10 13:13 index1.txt
-rw------- 1 root root 401M Sep 10 14:07 index20.txt
-rw------- 1 root root 412M Sep 10 14:10 index21.txt
-rw------- 1 root root 422M Sep 10 14:13 index22.txt
-rw------- 1 root root 392M Sep 10 14:16 index23.txt
-rw------- 1 root root 379M Sep 10 14:18 index24.txt
-rw------- 1 root root 178M Sep 10 14:21 index25.txt
-rw------- 1 root root  80M Sep 10 14:23 index26.txt
-rw------- 1 root root  94M Sep 10 1

In [ ]:
# Error : Because more than one '==' occuring in the title of documents
# f = open(doc_title_path,"r")
# for line in f:
#     docID,titleMap = line.split("==")
#     doc_title_map[docID] = titleMap
#     total_docs += 1

In [ ]:
# find() function too slow (~1 sec per word)
# def find_posting(path, start_flag, word):
#     fp = open(path,"r")
#     data = fp.read()
#     if start_flag:
#         start_index = data.find(word + "=")
#     else:
#         start_index = data.find("\n" + word + "=")
#     end_index = data.find("\n", start_index + 1)
#     return data[start_index : end_index]